In [1]:
# ปรับแต่งค่าการทำงานของ Spark
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("KafkaSubscribe").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1000m").\
        config("spark.executor.cores", "2").\
        config("spark.cores.max", "6").\
        config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0").\
        getOrCreate()



Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-032afaa6-ef15-4520-aadb-7908391af201;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 638ms :: artifacts dl 13

In [2]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "10.128.0.12:9092") \
  .option("subscribe", "quickstart-events") \
  .option("group.id", "Aekanun-Spark-App") \
  .load()

In [3]:
from pyspark.sql import functions as sparkf

In [4]:
# เลือกเฉพาะคอลัมน์ที่ต้องการ
df = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# แปลงข้อมูล JSON เป็นคอลัมน์ใน DataFrame
df = df.withColumn("data", sparkf.from_json("value", "struct<locationId:string, location:string, parameter:string, value:string, date:struct<utc:string, local:string>, unit:string, coordinates:struct<latitude:double, longitude:double>, country:string, city:string, isMobile:string, isAnalysis:string, entity:string, sensorType:string>"))

# แยกคอลัมน์ใน data เป็นคอลัมน์แยก
df = df.select("key", "data.*")

df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- date: struct (nullable = true)
 |    |-- utc: string (nullable = true)
 |    |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)



In [ ]:
query = df.writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='5 seconds') \
    .option("numRows", 3) \
    .start()

query.awaitTermination()


23/06/17 06:34:51 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c69e6781-e5f4-4735-b4d9-97c4231a6dc7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+----------+--------+---------+-----+----+----+-----------+-------+----+--------+----------+------+----------+
|key|locationId|location|parameter|value|date|unit|coordinates|country|city|isMobile|isAnalysis|entity|sensorType|
+---+----------+--------+---------+-----+----+----+-----------+-------+----+--------+----------+------+----------+
+---+----------+--------+---------+-----+----+----+-----------+-------+----+--------+----------+------+----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+----------+--------------------+---------+-----+----+-----+-----------+-------+----+--------+----------+--------------------+---------------+
| key|locationId|            location|parameter|value|date| unit|coordinates|country|city|isMobile|isAnalysis|              entity|     sensorType|
+----+----------+--------------------+---------+-----+----+-----+-----------+-------+----+--------+----------+--------------------+---------------+
|null|    225577|Bank of Ayuthaya ...|     pm25| 36.0|null|µg/m³|       null|     NA|    |   False|          |Governmental Orga...|reference grade|
|null|    225577|Bank of Ayuthaya ...|     pm25| 36.0|null|µg/m³|       null|     NA|    |   False|          |Governmental Orga...|reference grade|
|null|    225577|Bank of Ayuthaya ...|     pm25| 36.0|null|µg/m³|       null|     NA|    |   False|          |Governmental Orga...|reference grade|
+----+---------

-------------------------------------------
Batch: 11
-------------------------------------------
+----+----------+--------------------+---------+-----+----+-----+-----------+-------+----+--------+----------+--------------------+---------------+
| key|locationId|            location|parameter|value|date| unit|coordinates|country|city|isMobile|isAnalysis|              entity|     sensorType|
+----+----------+--------------------+---------+-----+----+-----+-----------+-------+----+--------+----------+--------------------+---------------+
|null|    225577|Bank of Ayuthaya ...|     pm25| 34.0|null|µg/m³|       null|     NA|    |   False|          |Governmental Orga...|reference grade|
|null|    225577|Bank of Ayuthaya ...|     pm25| 34.0|null|µg/m³|       null|     NA|    |   False|          |Governmental Orga...|reference grade|
|null|    225577|Bank of Ayuthaya ...|     pm25| 34.0|null|µg/m³|       null|     NA|    |   False|          |Governmental Orga...|reference grade|
+----+--------

-------------------------------------------
Batch: 12
-------------------------------------------
+----+----------+--------------------+---------+-----+----+-----+-----------+-------+----+--------+----------+--------------------+---------------+
| key|locationId|            location|parameter|value|date| unit|coordinates|country|city|isMobile|isAnalysis|              entity|     sensorType|
+----+----------+--------------------+---------+-----+----+-----+-----------+-------+----+--------+----------+--------------------+---------------+
|null|    225577|Bank of Ayuthaya ...|     pm25| 39.0|null|µg/m³|       null|     NA|    |   False|          |Governmental Orga...|reference grade|
|null|    225577|Bank of Ayuthaya ...|     pm25| 39.0|null|µg/m³|       null|     NA|    |   False|          |Governmental Orga...|reference grade|
|null|    225577|Bank of Ayuthaya ...|     pm25| 39.0|null|µg/m³|       null|     NA|    |   False|          |Governmental Orga...|reference grade|
+----+--------

In [ ]:
query.stop()